In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.1
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.1
tensorflow 2.0.1
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=7)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [5]:
layer = keras.layers.Dense(15, input_shape=(None, 5))
o = layer(tf.zeros([7, 5]))
print(o)

print(layer.variables)
print(layer.trainable_variables)


tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(7, 15), dtype=float32)
[<tf.Variable 'dense/kernel:0' shape=(5, 15) dtype=float32, numpy=
array([[ 0.01223534, -0.45291167,  0.18318945, -0.16670635, -0.09949532,
         0.2583    , -0.3556071 , -0.48200285, -0.1976046 , -0.5250265 ,
        -0.12405175, -0.07401097,  0.40203083,  0.2559693 , -0.2833841 ],
       [ 0.35402006,  0.00886136,  0.40201676, -0.29953578,  0.19633478,
         0.02911758, -0.4715309 , -0.27086392,  0.41890556,  0.34297723,
         0.13851571, -0.4268198 ,  0.21006691, -0.24516097,  0.08709306],
       [-0.08919147, -0.39838403,  0.05743176,  0.39168352,  0.52539825,
         0.40276408,  0.06958604,  0.3300

In [12]:
# tf.nn.softplus:   log(1+e^x),    比relu平滑，不会出现折点,  x> 0 时，约等于ｘ,　ｘ小于０时则很接近于0
customized_softplus = keras.layers.Lambda(lambda x : tf.nn.softplus(x))
print(customized_softplus([-10., -5., 0., 5., 10.]))

tf.Tensor([4.5417706e-05 6.7153489e-03 6.9314718e-01 5.0067153e+00 1.0000046e+01], shape=(5,), dtype=float32)


In [11]:
def customized_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        # x * W + b         input: x->[None, a],  W->[a, b]   output: [None, b]
        self.kernel = self.add_weight(name="kernel",
                                                      shape = (input_shape[1], self.units),
                                                      initializer = "uniform",
                                                      trainable = True
                                                     )
        self.bias = self.add_weight(name = "bias",
                                   shape = (self.units,),
                                   initializer = "zeros",
                                   trainable = True)
        
    def call(self, x):
        return self.activation(x @ self.kernel + self.bias)

model = keras.models.Sequential([
    #keras.layers.Dense(30, activation="relu", input_shape=x_train.shape[1:]),
    #keras.layers.Dense(1)
    CustomizedDenseLayer(30, activation="relu", input_shape=x_train.shape[1:]),
    CustomizedDenseLayer(1),
    customized_softplus,
    # CustomizedDenseLayer(1, "softplus")
    # CustomizedDenseLayer(1), keras.layers.Activation("softplus")
])
model.summary()
model.compile(loss=customized_loss, optimizer="sgd", metrics=["mean_squared_error"])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
customized_dense_layer_6 (Cu (None, 30)                270       
_________________________________________________________________
customized_dense_layer_7 (Cu (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [8]:
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
history = model.fit(x_train_scaled, y_train, validation_data=(x_valid_scaled, y_valid),
                   epochs=100, callbacks=callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 72us/sample - loss: 1.1971 - mean_squared_error: 1.1971 - val_loss: 0.6445 - val_mean_squared_error: 0.6445
Epoch 2/100
11610/11610 [==============================] - 0s 35us/sample - loss: 0.5546 - mean_squared_error: 0.5546 - val_loss: 0.5504 - val_mean_squared_error: 0.5504
Epoch 3/100
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4866 - mean_squared_error: 0.4866 - val_loss: 0.4926 - val_mean_squared_error: 0.4926
Epoch 4/100
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4637 - mean_squared_error: 0.4637 - val_loss: 0.5055 - val_mean_squared_error: 0.5055
Epoch 5/100
11610/11610 [==============================] - 0s 37us/sample - loss: 0.4587 - mean_squared_error: 0.4587 - val_loss: 0.4828 - val_mean_squared_error: 0.4828
Epoch 6/100
11610/11610 [==============================] - 1s 46us/sample - loss: 0.4338 - mean_squar

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=[8, 5])
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test)